说明：这就是一个简单的scikit-learn工具的使用，但不同的是训练数据集1000个，而测试数据有9000个，二分类问题。

同时，这边的数据都是数值，并且都已经标准化过，我们先来加载一下数据看看。

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
def loadata(file_train, file_label, file_test):
    X_train = pd.read_csv(file_train, header=None)
    y_train = pd.read_csv(file_label, header=None)
    X_test = pd.read_csv(file_test, header=None)
    return X_train, y_train, X_test

In [3]:
X_train, y_train, X_test = loadata('./数据集/train.csv', './数据集/trainLabels.csv', './数据集/test.csv')
y_train = np.array(y_train).ravel()

这边对于模型的选择，尝试使用LR，SVM，RF，GBDT，XGBOOST

In [4]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import xgboost as xgb

In [5]:
pf = PolynomialFeatures(2)
X_train = pf.fit_transform(X_train)

In [6]:
Xtrain, Xval, ytrain, yval = train_test_split(X_train, y_train)

In [ ]:
lr = LogisticRegression()
gscv_lr = GridSearchCV(lr, {'penalty': ['l1', 'l2'],
                            'tol': [1e-4, 1e-5, 1e-6],
                            'C': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
                            'random_state': range(501),
                            'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
                            'max_iter': [100, 200, 500, 1000, 2000, 5000]}, cv=10, n_jobs=-1)
gscv_lr.fit(Xtrain, ytrain)
lr = LogisticRegression(penalty=gscv.best_params_['penalty'], tol=gscv.best_params_['tol'],
                        C=gscv.best_params_['C'], random_state=gscv.best_params_['random_state'],
                        solver=gscv.best_params_['solver'], max_iter=gscv.best_params_['max_iter'])
lr.fit(Xtrain, ytrain)

In [ ]:
lr.score(Xval, yval)

In [ ]:
svm_c = SVC()
gscv_svc = GridSearchCV(svm_c, {'C': [0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
                                'kernel': ['linear', 'rbf'],
                                'tol': [1e-3, 1e-4, 1e-5, 1e-6],
                                'random_state': range(101)}, cv=10, n_jobs=-1)
gscv_svc.fit(Xtrain, ytrain)
svm_c = SVC(C=gscv.best_params_['C'], kernel=gscv.best_params_['kernel'],
            tol=gscv.best_params_['tol'], random_state=gscv.best_params_['random_state'])
svm_c.fit(Xtrain, ytrain)

In [ ]:
svm_c.score(Xval, yval)

In [ ]:
rfc = RandomForestClassifier()
gscv_rfc = GridSearchCV(rfc, {'n_estimators': [50, 80, 100, 200],
                              'max_depth': [3, 10, 30, 50],
                              'min_samples_split': [2, 5, 10],
                              'min_samples_leaf': [1, 2, 5],
                              'random_state': range(101)}, cv=10, n_jobs=-1)
gscv_rfc.fit(Xtrain, ytrain)
rfc = RandomForestClassifier(n_estimators=gscv.best_params_['n_estimators'],
                             max_depth=gscv.best_params_['max_depth'],
                             min_samples_split=gscv.best_params_['min_samples_split'],
                             min_samples_leaf=gscv.best_params_['min_samples_leaf'],
                             random_state=gscv.best_params_['random_state'])
rfc.fit(Xtrain, ytrain)

In [ ]:
rfc.score(Xval, yval)

In [ ]:
gbdt = GradientBoostingClassifier()
gscv_gbdt = GridSearchCV(gbdt, {'n_estimators': [50, 80, 100, 200],
                                'max_depth': [3, 5, 10, 30, 50],
                                'learning_rate': [0.05, 0.1, 0.2],
                                'min_samples_split': [2, 5, 10],
                                'min_samples_leaf': [1, 2, 5],
                                'subsample': [0.6, 0.8, 1.0],
                                'random_state': range(101)}, cv=10, n_jobs=-1)
gscv_gbdt.fit(Xtrain, ytrain)
gbdt = GradientBoostingClassifier(loss='exponential',
                                  n_estimators=gscv.best_params_['n_estimators'],
                                  max_depth=gscv.best_params_['max_depth'],
                                  learning_rate=gscv.best_params_['learning_rate'],
                                  min_samples_split=gscv.best_params_['min_samples_split'],
                                  min_samples_leaf=gscv.best_params_['min_samples_leaf'],
                                  subsample=gscv.best_params_['subsample'],
                                  random_state=gscv.best_params_['random_state'])
gbdt.fit(Xtrain, ytrain)

In [ ]:
gbdt.score(Xval, yval)

In [ ]:
xgbc = xgb.XGBClassifier()
gscv_xgbc = GridSearchCV(xgbc, {'n_estimators': [50, 80, 100, 200],
                                'max_depth': [3, 5, 10, 30, 50],
                                'learning_rate': [0.05, 0.1, 0.2],
                                'min_child_weight': [3, 5, 7, 9],
                                'subsample': [0.6, 0.8, 1.0],
                                'colsample_bytree': [0.6, 0.8, 0.1],
                                'reg_lambda': [0.01, 0.05, 0.1, 0.5, 1.0],
                                'reg_alpha': [0, 0.1, 0.5, 1.0],
                                'random_state': range(101)}, cv=10, n_jobs=-1)
gscv_xgbc.fit(Xtrain, ytrain)
xgbc = xgb.XGBClassifier(n_estimators=gscv_xgbc.best_params_['n_estimators'],
                         max_depth=gscv_xgbc.best_params_['max_depth'],
                         learning_rate=gscv_xgbc.best_params_['learning_rate'],
                         min_child_weight=gscv_xgbc.best_params_['min_child_weight'],
                         subsample=gscv_xgbc.best_params_['subsample'],
                         colsample_bytree=gscv_xgbc.best_params_['colsample_bytree'],
                         reg_lambda=gscv_xgbc.best_params_['reg_lambda'],
                         reg_alpha=gscv_xgbc.best_params_['reg_alpha'],
                         random_state=gscv_xgbc.best_params_['random_state'])
xgbc.fit(Xtrain, ytrain)

In [ ]:
xgbc.score(Xval, yval)

In [ ]:
import tpot

In [ ]:
tpotc = tpot.TPOTClassifier(n_jobs=-1, early_stop=True, verbosity=2)
tpotc.fit(X_train, y_train)

In [ ]:
tpotc.predict(X_test)